In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
try:
    from torch_geometric.nn import GCNConv
except ImportError:
    raise ImportError("PyTorch Geometric not found. Install with: pip install torch-geometric")
import numpy as np
import matplotlib.pyplot as plt

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Define the Knowledge Graph for a constrained pendulum
def create_pendulum_graph():
    """Create a KG for a pendulum: nodes = [pivot, mass], edges = [gravity, constraint]."""
    num_nodes = 2
    node_features = torch.tensor([[0.0, 0.0], [1.0, 0.0]], dtype=torch.float)  # [position, momentum]
    edge_index = torch.tensor([[0, 1], [1, 0]], dtype=torch.long).t()  # Bidirectional edges
    edge_attr = torch.tensor([[9.81], [1.0]], dtype=torch.float)  # [gravity, constraint]
    return node_features, edge_index, edge_attr

# Physics-Informed GNN Model
class PIML_GNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(PIML_GNN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)
        self.fc = nn.Linear(output_dim, output_dim)
    
    def forward(self, x, edge_index, edge_attr):
        """Forward pass: GCN with physics-informed constraints."""
        x = F.relu(self.conv1(x, edge_index, edge_attr))
        x = self.conv2(x, edge_index, edge_attr)
        x = self.fc(x)
        return x

# DAE-based physics loss for port-Hamiltonian system
def physics_loss(pred, x, edge_index, edge_attr, l=1.0, m=1.0, g=9.81):
    """Physics loss enforcing DAE constraints and energy conservation."""
    q, p = pred[:, 0], pred[:, 1]  # Position, momentum
    H = (p**2)/(2*m) + m*g*l*(1 - torch.cos(q))  # Hamiltonian
    dq_dt = p / m  # dq/dt = p/m
    dp_dt = -m * g * l * torch.sin(q)  # dp/dt = -mgl*sin(q)
    dynamics_loss = torch.mean((pred[:, 0] - dq_dt)**2 + (pred[:, 1] - dp_dt)**2)
    energy_loss = torch.mean((H - torch.mean(H))**2)
    return dynamics_loss + 0.1 * energy_loss, H

# Generate synthetic pendulum dataset
def generate_pendulum_data(num_samples=1000, t_max=10.0):
    """Generate synthetic data for pendulum dynamics."""
    t = np.linspace(0, t_max, num_samples)
    q = np.sin(t)  # Approximate position
    p = np.cos(t)  # Approximate momentum
    return torch.tensor(np.stack([q, p], axis=1), dtype=torch.float), t

# Plot predictions and energy
def plot_results(pred, true_data, t, H, epoch):
    """Plot predicted vs. true position, momentum, and energy."""
    pred = pred.detach().numpy()
    true_data = true_data.numpy()
    H = H.detach().numpy()
    
    plt.figure(figsize=(12, 8))
    plt.subplot(3, 1, 1)
    plt.plot(t[:2], pred[:, 0], label='Predicted Position', color='#1f77b4')
    plt.plot(t[:2], true_data[:, 0], label='True Position', linestyle='--', color='#ff7f0e')
    plt.title(f'Position (Epoch {epoch})')
    plt.legend()
    
    plt.subplot(3, 1, 2)
    plt.plot(t[:2], pred[:, 1], label='Predicted Momentum', color='#1f77b4')
    plt.plot(t[:2], true_data[:, 1], label='True Momentum', linestyle='--', color='#ff7f0e')
    plt.title('Momentum')
    plt.legend()
    
    plt.subplot(3, 1, 3)
    plt.plot(t[:2], H, label='Hamiltonian (Energy)', color='#1f77b4')
    plt.title('Energy Conservation')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig(f'pendulum_results_epoch_{epoch}.png')
    plt.close()

# Plot loss trends
def plot_losses(losses, data_losses, phys_losses):
    """Plot total loss, data loss, and physics loss over epochs."""
    plt.figure(figsize=(10, 6))
    epochs = range(0, len(losses) * 20, 20)
    plt.plot(epochs, losses, label='Total Loss', color='#1f77b4')
    plt.plot(epochs, data_losses, label='Data Loss', color='#ff7f0e')
    plt.plot(epochs, phys_losses, label='Physics Loss', color='#2ca02c')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Loss Trends')
    plt.legend()
    plt.grid(True)
    plt.savefig('loss_trends.png')
    plt.close()

# Training loop
def train_piml_gnn():
    """Train PIML-GNN with data and physics-informed loss."""
    input_dim, hidden_dim, output_dim = 2, 16, 2
    lr, epochs = 0.01, 200
    lambda_physics = 1.0

    model = PIML_GNN(input_dim, hidden_dim, output_dim)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    node_features, edge_index, edge_attr = create_pendulum_graph()
    data, t = generate_pendulum_data()
    
    losses, data_losses, phys_losses = [], [], []
    
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        pred = model(node_features, edge_index, edge_attr)
        data_loss = F.mse_loss(pred, data[:2])  # Match initial conditions
        phys_loss, H = physics_loss(pred, node_features, edge_index, edge_attr)
        loss = data_loss + lambda_physics * phys_loss
        loss.backward()
        optimizer.step()
        
        if epoch % 20 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item():.4f}, Data Loss: {data_loss.item():.4f}, Physics Loss: {phys_loss.item():.4f}")
            plot_results(pred, data[:2], t, H, epoch)
            losses.append(loss.item())
            data_losses.append(data_loss.item())
            phys_losses.append(phys_loss.item())
    
    plot_losses(losses, data_losses, phys_losses)

if __name__ == "__main__":
    try:
        train_piml_gnn()
    except Exception as e:
        print(f"Error during training: {e}")

Epoch 0, Loss: 29.3870, Data Loss: 0.3762, Physics Loss: 29.0108
Epoch 20, Loss: 3.0145, Data Loss: 0.0430, Physics Loss: 2.9715
Epoch 40, Loss: 1.5079, Data Loss: 0.0726, Physics Loss: 1.4353
Epoch 60, Loss: 0.8319, Data Loss: 0.1284, Physics Loss: 0.7035
Epoch 80, Loss: 0.5294, Data Loss: 0.1758, Physics Loss: 0.3536
Epoch 100, Loss: 0.4140, Data Loss: 0.2129, Physics Loss: 0.2012
Epoch 120, Loss: 0.3809, Data Loss: 0.2339, Physics Loss: 0.1470
Epoch 140, Loss: 0.3659, Data Loss: 0.2428, Physics Loss: 0.1231
Epoch 160, Loss: 0.3584, Data Loss: 0.2464, Physics Loss: 0.1120
Epoch 180, Loss: 0.3554, Data Loss: 0.2483, Physics Loss: 0.1072
